<img src="https://www.10botics.com/images/logo_website_700x200-300x86.png" width="300"/>

# 📸 任務一：確認裝備 (Camera Check) 

在我們開始訓練 AI 之前，我們必須先確認你的 Raspberry Pi 5 的「眼睛」是否看得清楚。

### 這個程式會做什麼？
1. 啟動你的相機。
2. 在這個網頁上即時顯示畫面。
3. 按下按鈕可以拍照並關閉相機。

In [ ]:
import cv2
import time
import ipywidgets as widgets
from IPython.display import display
from picamera2 import Picamera2
import logging
import threading

# 設定 picamera2 的日誌等級，只顯示錯誤 (讓畫面乾淨點)
Picamera2.set_logging(logging.ERROR)

# 定義相機迴圈函式
def stream_video_minimal(stop_button, image_widget):
    # 'try...finally' 確保相機被正確釋放，不會卡住
    try:
        # 'with' 陳述式會自動開啟和關閉相機
        with Picamera2() as picam2:
            # 建立一個 640x480 的相機設定 (這會使用預設的 RGBA 格式)
            config = picam2.create_preview_configuration(main={"size": (640, 480)})
            # 套用設定
            picam2.configure(config)
            # 啟動相機
            picam2.start()
            
            # 當停止按鈕沒有被按下時，持續執行
            while not stop_button.value:
                # 1. 從相機擷取一個畫面 (預設是 RGBA 格式)
                frame = picam2.capture_array()
                
                # 2. 將顏色從 RGBA 轉換為 BGR (OpenCV 預設格式)
                # 這一步會把 紅(R) 和 藍(B) 對調，修復「阿凡達」膚色問題
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)
                
                # 3. 翻轉影像
                # 如果鏡頭是倒的，請把 1 改成 -1 (左右翻轉) 或註解掉這行
                frame_flipped = cv2.flip(frame_bgr, 1)
                
                # 4. 將影像編碼為 JPEG 格式顯示
                # quality=70 可以讓傳輸快一點
                _, buffer = cv2.imencode('.jpeg', frame_flipped, [int(cv2.IMWRITE_JPEG_QUALITY), 70])
                
                # 更新 widget 顯示
                image_widget.value = buffer.tobytes()
                
                # 短暫暫停，釋放 CPU
                time.sleep(0.01)

    except Exception as e:
        print(f"發生錯誤: {e}")
        
    finally:
        # 當迴圈停止 (按鈕被按下) 時，清空畫面
        image_widget.value = b''
        print("相機已關閉")

# --- 主程式 ---

# 1. 建立停止按鈕
stopButton = widgets.ToggleButton(
    value=False, description='停止相機', button_style='danger', icon='square'
)

# 2. 建立顯示影像的區域 (640x480)
image_widget = widgets.Image(format='jpeg', width=640, height=480)

# 3. 顯示介面
display(widgets.VBox([image_widget, stopButton]))

# 4. 使用執行緒 (Thread) 啟動相機
# 這樣才不會讓你的 Jupyter Notebook 卡死
thread = threading.Thread(
    target=stream_video_minimal, 
    args=(stopButton, image_widget)
)
thread.start()

### ❓ 遇到問題了嗎？

* **黑畫面或錯誤訊息？**
    * 請檢查 USB 攝像頭或 Pi Camera 是否插緊。
    * 嘗試重新啟動這個 Kernel (上方選單 Kernel -> Restart Kernel)。
    * 如果還是不行，舉手求救！🙋‍♂️

<hr/>

## Congratulation! You have finished this chapter.

This jupyter notebook is created by 10Botics. <br>
For permission to use in school, please contact info@10botics.com <br>
All rights reserved. 2024.